
# Notebook CoT aplicada

Clase aplicada de Zero-shot CoT + Few-shot CoT con feedback loop para un agente conversacional coqueto y respetuoso.


In [ ]:

from __future__ import annotations

import json
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

ROOT = Path.cwd()
sys.path.append(str((ROOT / "02-prompting").resolve()))
from common.rubrica import evaluar_salida

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Falta OPENAI_API_KEY en .env")

model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
client = OpenAI(api_key=api_key)
print(f"Modelo activo: {model}")


In [ ]:

perfil = {
    "tipo_persona": "product manager de salud digital",
    "gustos": ["running", "cafes tranquilos", "podcasts de tecnologia"],
    "estilo": "directo y curioso",
    "contexto": "primera conversacion despues de un match",
}

system_prompt = (
    "Eres coach conversacional. Tono coqueto y respetuoso. "
    "Prohibido lenguaje explicito, manipulador o de presion. "
    "No uses emojis."
)

user_prompt = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Haz razonamiento en 4 pasos y luego entrega JSON exacto:
{{
  "chain_of_thought": ["...", "...", "...", "..."],
  "opener": "...",
  "follow_up": "...",
  "tone_notes": ["...", "..."],
  "avoid": ["...", "..."]
}}
"""

resp = client.chat.completions.create(
    model=model,
    temperature=0.6,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

base = json.loads(resp.choices[0].message.content)
print(json.dumps(base, ensure_ascii=False, indent=2))


In [ ]:

evaluacion_base = evaluar_salida(perfil, base)
print(json.dumps(evaluacion_base, ensure_ascii=False, indent=2))


In [ ]:

prompt_mejora = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Salida actual:
{json.dumps(base, ensure_ascii=False, indent=2)}

Rubrica:
{json.dumps(evaluacion_base, ensure_ascii=False, indent=2)}

Mejora la respuesta para subir el promedio de la rubrica.
No uses emojis.
Devuelve JSON con la misma estructura.
"""

resp2 = client.chat.completions.create(
    model=model,
    temperature=0.3,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres un revisor de calidad de prompting."},
        {"role": "user", "content": prompt_mejora},
    ],
)

mejorada = json.loads(resp2.choices[0].message.content)
evaluacion_mejorada = evaluar_salida(perfil, mejorada)

print("Version mejorada")
print(json.dumps(mejorada, ensure_ascii=False, indent=2))
print("Evaluacion mejorada")
print(json.dumps(evaluacion_mejorada, ensure_ascii=False, indent=2))
